# Домашнее задание №1

In [1]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Get the same results each time
np.random.seed(0)


# Load the training data
df = pd.read_csv("./data/data.csv")
comments = df["comment_text"]
target = (df["target"]>0.7).astype(int)

In [3]:
# Задание 1
# Делим выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.3)

In [4]:
# Задание 2
# Векторизируем комментарии
vect = CountVectorizer()
train = vect.fit_transform(X_train['comment_text'])
test = vect.transform(X_test['comment_text'])

In [5]:
# Задание 3
# Строим модель логистической регрессии 
log = LogisticRegression(max_iter=2000)
log.fit(train, y_train)
predict = log.predict(test)
accuracy_score(y_test, predict)

0.9287888232921419

In [6]:
# Задание 4
# Функция анализа пользовательского комментария
def analyze_comment_proba(comment, vect, model):
    vector = vect.transform([comment])
    predict = model.predict_proba(vector)
    return predict[0][1]

def predict_comment_toxic(comment, vect, model):
    vector = vect.transform([comment])
    predict = model.predict(vector)
    return predict[0]

In [10]:
# Задание 5
# Определение пользовательских комментариев
comment_1 = 'Apples are stupid'
comment_2 = 'I love apples'
print(f'Вероятность того, что комментарий {comment_1} токсичный: ',
      analyze_comment_proba(comment_1, vect, log).round(5))
print(f'Вероятность того, что комментарий {comment_2} токсичный: ',
      analyze_comment_proba(comment_2, vect, log).round(5))
print(f'Комментарий {comment_1} ', 'токсичный' if predict_comment_toxic(comment_1, vect, log) else 'не токсичный')
print(f'Комментарий {comment_2} ', 'токсичный' if predict_comment_toxic(comment_2, vect, log) else 'не токсичный')

Вероятность того, что комментарий Apples are stupid токсичный:  0.99945
Вероятность того, что комментарий I love apples токсичный:  0.09842
Комментарий Apples are stupid  токсичный
Комментарий I love apples  не токсичный


In [ ]:
# Задание 6
# Определение самых токсичных слов для модели
log_coef = log.coef_[0]
word_coef = {}
for word, idx in vect.vocabulary_.items():
    word_coef[word] = log_coef[idx]

sorted_word_coef = sorted(word_coef.items(), key=lambda x: abs(x[1]), reverse=True)

positive_coefs = sorted(word_coef.items(), key=lambda x: x[1], reverse=True)
print("\nТоп-10 слов, указывающих на класс 1:")
for word, coef in positive_coefs[:10]:
    print(f"{word}")



Топ-10 слов, указывающих на класс 1:
stupid
idiot
idiots
stupidity
idiotic
crap
dumb
pathetic
morons
moron


Задание 7

Все слова, которые перечислены в списке выше подходят под определение токсичных, никаких неожиданных слов обнаружено не было.

In [20]:
# Задание 8
# Проверка на этичность

ethic_comment_1 = "I have a christian friend"
ethic_comment_2 = "I have a muslim friend"
ethic_comment_3 = "I have a white friend"
ethic_comment_4 = "I have a black friend"

print(f'Вероятность того, что комментарий {ethic_comment_1} токсичный: ',
      analyze_comment_proba(ethic_comment_1, vect, log).round(5))
print(f'Вероятность того, что комментарий {ethic_comment_2} токсичный: ',
      analyze_comment_proba(ethic_comment_2, vect, log).round(5))
print(f'Вероятность того, что комментарий {ethic_comment_3} токсичный: ',
      analyze_comment_proba(ethic_comment_3, vect, log).round(5))
print(f'Вероятность того, что комментарий {ethic_comment_4} токсичный: ',
      analyze_comment_proba(ethic_comment_4, vect, log).round(5))

print(f'Комментарий {ethic_comment_1} ', 'токсичный' if predict_comment_toxic(ethic_comment_1, vect, log) else 'не токсичный')
print(f'Комментарий {ethic_comment_2} ', 'токсичный' if predict_comment_toxic(ethic_comment_2, vect, log) else 'не токсичный')
print(f'Комментарий {ethic_comment_3} ', 'токсичный' if predict_comment_toxic(ethic_comment_3, vect, log) else 'не токсичный')
print(f'Комментарий {ethic_comment_4} ', 'токсичный' if predict_comment_toxic(ethic_comment_4, vect, log) else 'не токсичный')

Вероятность того, что комментарий I have a christian friend токсичный:  0.12294
Вероятность того, что комментарий I have a muslim friend токсичный:  0.4482
Вероятность того, что комментарий I have a white friend токсичный:  0.32911
Вероятность того, что комментарий I have a black friend токсичный:  0.5163
Комментарий I have a christian friend  не токсичный
Комментарий I have a muslim friend  не токсичный
Комментарий I have a white friend  не токсичный
Комментарий I have a black friend  токсичный


В результате видим, что комментарий I have a christian friend наименее токсичный из всех, хотя смысловая нагрузка во всех комментариях одинкова (а комментарий I have a black friend вообще пометили, как токсичный). Следовательно, у модели есть bias. 

Задание 9

В тексте дано условие о том, что сообщество, представленное в выборке, исламофобно. Следовательно, это предвзятость социально-историческую предвзятость, которая связана с самими людьми, которые писали данные комментарии.

Задание 10

Идея 1: кажется, что самым простым и эффективным решением будет добавить в выборку некоторые положительные комментарии об исламе (если в остальном модель нас полностью устраивает). Либо можно исключить часть токсичных комментариев, связанных с исламом.

Идея 2: изменить модель с обычной логистической регрессии на более сложную, например, трансформерные модели. Тогда модель сможет улавливать контекст в словах, а не просто анализировать предложение по ключевым словам. Например, модель BERT.